Voici le code final intégrant nos améliorations ! A vous de jouer ;)

In [ ]:
class Ordre:
    def __init__(self, identifiant, type_ordre, quantite, prix):
        self.identifiant = identifiant
        self.type_ordre = type_ordre
        self.quantite = quantite
        self.prix = prix

class CarnetOrdres:
    def __init__(self):
        self.ordres_achat = []
        self.ordres_vente = []

    def ajouter_ordre(self, ordre):
        if ordre.type_ordre == 'achat':
            self.ordres_achat.append(ordre)
            self.ordres_achat.sort(key=lambda x: (-x.prix, x.identifiant))
        elif ordre.type_ordre == 'vente':
            self.ordres_vente.append(ordre)
            self.ordres_vente.sort(key=lambda x: (x.prix, x.identifiant))
        else:
            raise ValueError("Le type d'ordre doit être 'achat' ou 'vente'")
        self.executer_ordres()

    def executer_ordres(self):
        while self.ordres_achat and self.ordres_vente and self.ordres_achat[0].prix >= self.ordres_vente[0].prix:
            ordre_achat = self.ordres_achat[0]
            ordre_vente = self.ordres_vente[0]
            quantite_executee = min(ordre_achat.quantite, ordre_vente.quantite)
            prix_execute = ordre_vente.prix
            
            print(f"Ordre exécuté : {quantite_executee} actions à {prix_execute} € par action")
            
            ordre_achat.quantite -= quantite_executee
            ordre_vente.quantite -= quantite_executee
            
            if ordre_achat.quantite == 0:
                self.ordres_achat.pop(0)
            if ordre_vente.quantite == 0:
                self.ordres_vente.pop(0)

    def determine_fixing_price(self):
        if not self.ordres_achat or not self.ordres_vente:
            print("Pas assez d'ordres pour déterminer le prix de fixing.")
            return None

        executable_volumes = {}
        for buy_order in self.ordres_achat:
            for sell_order in self.ordres_vente:
                if buy_order.price >= sell_order.price:
                    price = sell_order.price
                    executable_volumes[price] = executable_volumes.get(price, 0) + min(buy_order.quantity, sell_order.quantity)
                    
        if not executable_volumes:
            print("Aucun ordre correspondant pour le fixing.")
            return None

        fixing_price = max(executable_volumes, key=executable_volumes.get)
        print(f"Prix de fixing déterminé : {fixing_price} avec un volume total : {executable_volumes[fixing_price]}")
        return fixing_price

    def perform_fixing(self):
        fixing_price = self.determine_fixing_price()
        if fixing_price is not None:
            self.execute_orders_at_fixing_price(fixing_price)

    def execute_orders_at_fixing_price(self, fixing_price):
        pass

def main():
    carnet_ordres = CarnetOrdres()
    while True:
        print("\n1. Ajouter un ordre\n2. Effectuer un fixing\n3. Voir le carnet d'ordres\n4. Quitter")
        action = input("Choisissez une action : ")
        if action == '1':
            identifiant = int(input("ID de l'ordre : "))
            type_ordre = input("Type d'ordre (achat/vente) : ")
            quantite = int(input("Quantité : "))
            prix = float(input("Prix : "))
            ordre = Ordre(identifiant, type_ordre, quantite, prix)
            carnet_ordres.ajouter_ordre(ordre)
        elif action == '2':
            carnet_ordres.perform_fixing()
        elif action == '3':
            print("Ordres d'achat :")
            for order in carnet_ordres.ordres_achat:
                print(f"ID : {order.identifiant}, Prix : {order.prix}, Quantité : {order.quantite}")
            print("Ordres de vente :")
            for order in carnet_ordres.ordres_vente:
                print(f"ID : {order.identifiant}, Prix : {order.prix}, Quantité : {order.quantite}")
        elif action == '4':
            break
        else:
            print("Action invalide. Veuillez choisir une option valide.")

if __name__ == "__main__":
    main()